# PART 3 of 3. In this notebook, using a real world example, I detail how to use benchmarking to make smarter pricing decisions.

This is the third in a series of three articles related to pricing and pricing decision making. In the previous notebook we learned that our organization's price elasticity of revenue is .35. This means that a 1% increase in price will generate a .35% increase in revenue. So, if we increase prices across all stores by 3%, we will increase revenue by 1.05%.

Can we do better than this? Fortunately, we can. How? By catering pricing decisions so they are specific to each store. So, instead of increasing by 3% across the board, increase price by more than 3% in the locations that face a robust market and increase price by less than 3% in markets that are less robust. This will allow us to increase revenue by something greater than 1.05% with a 3% price increase.

So how do you know which markets can sustain a higher price increase? Well, there are many ways to estimate this. The technique we will walk through in this article is relatively low-tech, but extremely effective.

To give credit where it is due, I didn't invent this. I've used this technique in many applications for many years, but I learned it from some incredibly smart people at GTE Laboratories almost twenty years ago. GTE hasn't existed for a long time. It was a great company.

Again, we'll be using Watson Studio, python, a Jupyter notebook, pandas, Plotly and Watson object storage.



# Import data and data transformations

First step is import your libraries and data.

In [ ]:
import sys
import types
import statsmodels.api as sm
import pandas as pd
import numpy as np
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

!pip install plotly --upgrade
import plotly
#PUT YOUR PLOTLY CREDENTIALS HERE
plotly.tools.set_credentials_file(username='XXXXXXX', api_key='XXXXXXX')
import plotly.plotly as py
import plotly.graph_objs as go

The next step is to upload the pricing data into object storage. To do this, click on the data icon (it is the one with two 1's and two 0's) in the northeast corner of the Watson Studio interface. 

Next, on the "Files" tab click on "Browse" where it says, "Drop your file here or browse your files to add a new file". 

From there, navigate to our data set "RETAIL_DATA.csv" and upload it to the cloud.

Once the file is uploaded it should appear in the data tab. From the drop down menu, select insert credentials. It should look something like this.

In [ ]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': 'XXXXXXX',
    'IAM_SERVICE_ID': 'XXXXXXX',
    'ENDPOINT': 'XXXXXXX',
    'IBM_AUTH_ENDPOINT': 'XXXXXXX',
    'BUCKET': 'XXXXXXX',
    'FILE': 'RETAIL_DATA.csv'
}


The next snippet of code will convert the file you just uploaded to object storage into a pandas data frame. Replace the credentials below with the credentials you inserted in the cell above.

In [ ]:

import sys
import types
import pandas as pd
from ibm_botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_e2f5521e5ce34913a7133599cf5a489b = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='XXXXXXX',
    ibm_auth_endpoint="XXXXXXX",
    config=Config(signature_version='oauth'),
    endpoint_url='XXXXXXX')

body = client_e2f5521e5ce34913a7133599cf5a489b.get_object(Bucket='XXXXXXX',Key='RETAIL_DATA.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_retail = pd.read_csv(body)
df_retail.head()


Here is a definition of each field.

STORE_ID - is a unique id specific to each retail outlet

PERCENTAGE_OF_RENTERS is the percentage of households surrounding the store that rent their housing.

PERCENTAGE_OF_CHILDREN is the percentage of households surrounding the store that have children.

AVERAGE_INCOME is the average annual income of the households surrounding the store.

AVERAGE_AGE_IN_YEARS is the average age of the head of household in the vicinity of the retail outlet.

AVERAGE_LENGTH_OF_RESIDENCE is an average of the time individuals surrounding the retail outlet have lived at their current address.

PERCENT_SPEAKING_SPANISH is the percentage of households surrounding the store that speak Spanish

PRICE is the average price across multiple items sold at the retail outlet.

QUANTITY is the number of items sold by the retail outlet in the last year.

REVENUE is the total revenue for the store in the last year.

Now, create new fields by taking the natural log of each variable.

In [ ]:
df_retail['LN_PRICE'] = np.log((df_retail.PRICE))
df_retail['LN_REVENUE'] = np.log((df_retail.REVENUE))
df_retail['LN_QUANTITY'] = np.log((df_retail.QUANTITY))
df_retail['LN_INCOME'] = np.log((df_retail.AVERAGE_INCOME))
df_retail['LN_AVERAGE_AGE_IN_YEARS'] = np.log((df_retail.AVERAGE_AGE_IN_YEARS))
df_retail['LN_AVERAGE_LENGTH_OF_RESIDENCE'] = np.log((df_retail.AVERAGE_LENGTH_OF_RESIDENCE))
df_retail['LN_PERCENT_SPEAKING_SPANISH'] = np.log((df_retail.PERCENT_SPEAKING_SPANISH))
df_retail['LN_PERCENT_HAVING_CHILDREN'] = np.log((df_retail.PERCENT_HAVING_CHILDREN))
df_retail['LN_PERCENTAGE_OF RENTERS'] = np.log((df_retail.PERCENTAGE_OF_RENTERS))

df_retail.head()


# Establishing a pricing benchmark

The first step in our exercise is to understand how each store's pricing compares to other stores. For example, does store 1234 have higher prices or lower prices compared to other stores? 

Obviously, there are many ways to do this. In this example, we will keep it simple. We will create a price index for each store that is equal to the price of the store divided by the average price for all stores.

Price Index(i) = Price of store(i) /Average Price of all Stores


Let's add the price index for each store to our data frame

In [ ]:
df_retail['loopydoopy']=1

wookie = pd.DataFrame(df_retail.groupby(['loopydoopy'])['PRICE'].mean())
wookie.reset_index(level=0, inplace=True)
wookie.columns=['loopydoopy','AVERAGE_PRICE']
df_retailx =df_retail.merge(wookie, on=['loopydoopy'], how='inner')
df_retailx['PRICE_INDEX']=df_retailx['PRICE']/df_retailx['AVERAGE_PRICE']
df_retailx=df_retailx.drop([ 'loopydoopy'], axis=1)
df_retailx.head()


The last column of data set is the new price index. Note that if the price index is greater than 1, the store has higher than average prices. If the price index is less than 1, the store has lower than average prices.

# Establishing a Revenue Benchmark

To benchmark revenue, we will use a regression model. 

One thing you have to understand about real-world retail data is that there is no level playing field. For example, Store 123 may have better sales than store 456. Store 123 wins, right? 

Maybe, maybe not. 

What if I told you that the Store 123 is in a high-income, rapidly growing part of town and store 456 is in an economically depressed neighborhood. You'd expect store 123 to do better, right? This is a common problem that business leaders face. How can you compare different retail outlets when they are not facing the same obstacles?

The easiest way is to exploit multivariate statistics. Let me show you want I mean.

First, we will build a regression model that uses market conditions to predict revenue. Note that we are not using Price or the natural log of price as a predictor variable.

In [ ]:
independentx = df_retailx[['LN_PERCENTAGE_OF RENTERS','LN_PERCENT_HAVING_CHILDREN','LN_INCOME',
                          'LN_PERCENT_SPEAKING_SPANISH']]
independent = sm.add_constant(independentx, prepend=False)
dependent=df_retail['LN_REVENUE']


In [ ]:
mod = sm.OLS(dependent, independent)

In [ ]:
res = mod.fit()

In [ ]:
print(res.summary())

An OLS regression will produce a residual and a predicted value for each observation.

A predicted value is a model prediction of dependent variable, in this case the natural log of revenue.

Another way to interpret the predicted variable is that it is the expected value of the dependent variable, given the independent variables.

In our case, this means that the predicted value is the natural log of revenue we would expect for each store, given the market conditions facing the store.

If you are fairly new to statistics, this is kind of an obtuse idea. If it totally doesn't make sense, I think it will when I give a concrete example in just a second.

Beyond Statistical modeling, we don't really care about the Natural Log of Revenue, so let's export the predicted values from the model to a new data frame and convert them to actual revenue, then append the new fields to our main data frame.


In [ ]:
df_ypred = pd.DataFrame(res.predict(independent))

df_ypred.columns = ['P_LN_REVENUE']
df_ypred['P_REVENUE']=np.exp(df_ypred['P_LN_REVENUE'])
df_ypred.head()

In [ ]:
df_retaily = pd.concat([df_retailx, df_ypred], axis=1)

df_retaily.head()

Now, let's keep the columns we need, trashing the ones we don't.

In [ ]:
df_benchmarking = df_retaily[['STORE_ID','PRICE','AVERAGE_PRICE','PRICE_INDEX',
                          'REVENUE', 'P_REVENUE']]
df_benchmarking.head()

Ok, STORE_ID 175217 is the first row in the table above. It's price is 7.50, which is pretty close to the average price of 7.47. It's price index is slightly greater than 1 at a 1.0032.

In the previous year, the store realized 713,280 dollars in revenue. Remember our interpretation of the predicted revenue from our model is the revenue the store SHOULD have realized given the conditions of its market.

So, given the level of income, Spanish speaking population and other market factors, the predicted revenue is what we would expect an average store to realize in that specific market.

In the previous year, given the market it faced, we would have expected store 175217 to realize about $566,468 in revenue. It actually did much better than that. It realized over 713,000 dollars in revenue. Based on this, we can say that store 175217 OVER-PERFORMED. That is, it did much better than expected, given the market it had to work with.

So, just to reiterate, the predicted revenue in this case is what we'd expect a store to do given its market. Revenue is what it actually did.

With these two key pieces of information, we can now create a revenue index to pair with our price index. To get the revenue index, we'll divide the actual revenue by predicted revenue.

Revenue Index(i)=(Revenue(i)/Predicted Revenue(i)).

If the Revenue Index is greater than 1, the store is OVER PERFORMING. If the Revenue Index is less than 1, the store is UNDER PERFORMING.


In [ ]:
dfz=pd.DataFrame((df_benchmarking['REVENUE'])/(df_benchmarking['P_REVENUE']))
dfz.columns = ['REVENUE_INDEX']
df_benchmarking = pd.concat([df_benchmarking, dfz], axis=1)
df_benchmarking.head()


# Visualizing and making better decisions around pricing.

Now we have an idea of how each store is priced, relative to other stores. We also have an idea of how each store is performing, given the market it faces. Combining these two pieces of information gives us a powerful decision-making tool. Let me show you want I mean visually in a chart.

In [ ]:
x1 = df_benchmarking['PRICE_INDEX']
y1 = df_benchmarking['REVENUE_INDEX']


trace = go.Scatter(
    x = x1,
    y = y1,
    mode = 'markers'
)

trace1 = go.Scatter(
    x = [1,1,1,1],
    y = [0.0,1,1.2,2.5],
       line = dict(
        color = ('#FF00FF'),
        width = 4,)

)
trace2 = go.Scatter(
    y = [1,1,1,1],
    x = [0.8,1,1.1,1.2],
       line = dict(
        color = ('#FF00FF'),
        width = 4,)

)

layout = go.Layout(
    title='Price-Revenue Matrix',
    xaxis=dict(
        title='Price Index',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Revenue Index',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    showlegend=False,
    annotations=[
        dict(
            x=1.1,
            y=0.5,
            xref='x',
            yref='y',
            text='Under-Performing and Relatively Expensive',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=0
        ),
        dict(
            y=0.5,
            x=0.9,
            xref='x',
            yref='y',
            text='Under-Performing and Relatively Inexpensive',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=0
        ),
        dict(
            y=1.5,
            x=0.9,
            xref='x',
            yref='y',
            text='Over-Performing and Relatively Inexpensive',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=0
        ),
         dict(
            y=1.5,
            x=1.1,
            xref='x',
            yref='y',
            text='Over-Performing and Relatively Expensive',
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=0
        ),
    ]
)
    
data=[trace,trace1,trace2]  
fig = go.Figure(data=data, layout=layout)

#plot_url = py.plot(fig, filename='styling-names')
py.iplot(fig, filename='shapes-lines')

In the chart above, the price index is represented by the x axis. The revenue index is represented by the y axis. Reference lines are added to represent a revenue index of 1 and a price index of 1.

To understand this chart, let's start with the northeast quadrant. In this portion of the graph, you'll find stores who are over performing with prices that are higher than average.

In the southeast quadrant, you will find stores who are under performing, with prices higher than average.

In the southwest quadrant, you will find stores that are under-performing with lower than average prices.

And finally, in the northwest quadrant, you will find stores that are over-performing with less than average prices.

# Pulling it all together and making some decisons

So, taking everything that we have learned in this exercise, what is the best course of action? 

Of course, it depends, but here is a logical next step that would make sense in many situations. 

Let's start with what we learned in our 2nd notebook, that the revenue elasticity of our stores is .35. This means an increase in prices will generate additional revenue. We don't want to go crazy, but I think an average 3% increase makes sense. Remember that inflation is about 2% a year, so a 3% increase is actually a 1% increase when your account for general inflation.

So, our first decision is to increase prices by an average of 3%.

Now, we need to decide how to allocate the price increase. We'll do this by splitting our stores in to three different groups.

Group 1 - a price increase of 4%.

Group 2 - a price increase of 2%.

Group 3 - a price increase of 0%.

For Group 1, it makes sense to select stores that are Under-Performing and Under-Priced. These are stores that should be doing much better and there is a really good chance that their lack of performance is from prices that are too low.

For Group 2, it makes sense to select stores that are Over-Performing. Remember that a 2% increase is really a price freeze when you take inflation into account. These stores are doing really well, just let them be.

For Group 3, it makes sense to select stores that are Under-Performing and Over-Priced. It makes sense that these stores may have performance issues because they are priced too high.


The code below will assign a price change to each store. To make the average be close to 3%, I adjusted the cut-offs defining what is high and low (1.25 for revenue and 1.05 for price). Note, I did this with a little trial and error.

Of course, you want to use sound business judgment to set your prices. In this case, I am working on the assumption that given the overall revenue elasticity, most stores should be priced higher. Also, remember that a 4% increase is really a 2% increase when you factor in inflation.

Group 1: If the revenue index is less than 1.25 and the price index is less than 1.05, the rate up is 4%. These stores are under-performing and are under-priced. They will get a price increase.

Group 2: If the revenue index is greater than 1.25 the rate-up is 2%. These stores are over-performing, so we will just let them be with a price increase the same as inflation.

Group 3: If the revenue index is less than 1.25 and the price index is greater than 1.05 the rate up is 0%. These stores are under-performing and they are over-priced. They get a rate hold, which is really a price decrease when you factor in inflation.


In [ ]:


df_benchmarking.loc[ (df_benchmarking.REVENUE_INDEX >= 1.25) & (df_benchmarking.PRICE_INDEX>= 1.05), 'PRICE_INCREASE' ] = 2
df_benchmarking.loc[ (df_benchmarking.REVENUE_INDEX >= 1.25) & (df_benchmarking.PRICE_INDEX< 1.05), 'PRICE_INCREASE' ] = 2
df_benchmarking.loc[ (df_benchmarking.REVENUE_INDEX <1.25) & (df_benchmarking.PRICE_INDEX< 1.05), 'PRICE_INCREASE' ] = 4
df_benchmarking.loc[ (df_benchmarking.REVENUE_INDEX <1.25) & (df_benchmarking.PRICE_INDEX>=1.05), 'PRICE_INCREASE' ] = 0

Note that we have achieved the goal of keeping the average price change at 3% (2.996% actually).

In [ ]:
df_benchmarking.describe()

Note that 50 stores will get a price increase of 0%. Forty-six will get a price increase of 2% and 195 will get a price increase of 4%.

In [ ]:
df_benchmarking.PRICE_INCREASE.value_counts(dropna = False).sort_index()